In [28]:
from datetime import datetime
import json
import urllib

from bs4 import BeautifulSoup

In [29]:
def getUrlSoup(url):
    # Get the page response.
    response = urllib.urlopen(url)
    # Read the page response into raw content.
    content = response.read()
    # Soup it.
    soup = BeautifulSoup(content)
    return soup


user_ids = ['ur51685054','ur2363449']

soup = getUrlSoup('http://www.imdb.com/user/' + user_ids[0] + '/')

---
### Reviews

Get all the reviews for this specific user.

In [31]:
reviews_block = (soup.body.find('div', attrs={'class': 'widget article reviews'})
                          .find('div', attrs={'class':'reviews'}))

# Create an empty reviews object.
reviews = []

for div in reviews_block.findAll('div',attrs={'class':'toggle-overflow'}):
    
    review = dict()
    
    review['movie_link'] = div.find('h3').find('a')['href']
    review['movie_name'] = div.find('h3').find('a').get_text()
    review['movie_year'] = div.find('h3').find('span').get_text().lstrip('(').rstrip(')')
    review['review_title'] = div.find('h4').get_text()
    
    review_date_str = div.find('div', attrs={'class':'details'}).get_text().split(' - ')[0]
    review['review_date'] = str(datetime.strptime(review_date_str, '%d %B %Y'))
    
    comment_paragraphs = div.findAll('p')
    review['review_content'] = ''
    for p in comment_paragraphs:
        review['review_content'] += p.get_text()
    
    reviews.append(review)

    
# Print the first two to see it worked.
print json.dumps(reviews[:2],indent=4)

[
    {
        "review_date": "2016-01-09 00:00:00", 
        "review_content": "Kill game more like kill me as this film is awful from the script to the acting, screen shots camera angles the only best part is when you see the credits as then you know its finished. Its the same old story of a bunch of teenagers who play pranks on others and would you believe it one of those pranks goes wrong and someone dies, 5 years later somebody starts killing the group members one by one. From the start you can figure out who the killer is and evan the kills are pretty stupid theirs not a lot of good things to say about this film and evan the harcore thriller/horror fans out there would not like this film. Overall kill game is a low budget very poor attempt at a horror film that's plot has been done more times than iv had Sunday dinners.", 
        "review_title": "this film will kill brain cells its that bad", 
        "movie_name": "Kill Game", 
        "movie_year": "2015", 
        "movie_lin

---
### Ratings

not iterating over all pages yet...

In [35]:
soup_user_ratings = getUrlSoup(urls[0] + 'ratings')

In [36]:
num_ratings = (soup_user_ratings.body.find('div', attrs={'class':'header'})
                                     .find('div', attrs={'class':'nav'})
                                     .find('div', attrs={'class':'desc'})['data-size'])

print num_ratings

344


In [44]:
ratings_block = soup_user_ratings.body.find('div', attrs={'class': 'list detail'})

ratings_blocks = ratings_block.findAll('div', attrs={'class':'list_item'})

ratings = []

for b in ratings_blocks:
    rating = dict()
    rating['movie_href'] = b.find('div',attrs={'class':'info'}).find('a')['href']
    rating['movie_name'] = b.find('div',attrs={'class':'info'}).find('a').get_text()
    rating['user_movie_rating'] = int(b.find('div',attrs={'class':'info'})
                                    .find('div',attrs={'class':'secondary quoted_rating'})
                                    .get_text().split('rated this')[1].rstrip('.').lstrip())
    ratings.append(rating)
    
print json.dumps(ratings[:3],indent=4)

[
    {
        "user_movie_rating": 9, 
        "movie_name": "The Big Short", 
        "movie_href": "/title/tt1596363/"
    }, 
    {
        "user_movie_rating": 2, 
        "movie_name": "Kill Game", 
        "movie_href": "/title/tt3024324/"
    }, 
    {
        "user_movie_rating": 3, 
        "movie_name": "Sisters", 
        "movie_href": "/title/tt1850457/"
    }
]
